In [2]:
from tqdm import tqdm
import os
import numpy as np

In [12]:
import requests
import json

req = requests.get('https://osi.ipf.msu.edu/piwebapi/streamsets/F1EmxYSRoDn9-066ETsCvigyBwR3FABHu96xGtiqCkxcOYdQSVBGLVBQSElTVERBVEFcQ0FMQ19URVNUXEFLRVJTIEhBTExcMDMyNlcx/recorded?StartTime=*-1186d&EndTime=*-0d&MaxCount=120000')

In [25]:
pd.to_datetime(req.json()['Items'][0]['Items'][-1]['Timestamp']).date()
date(pd.to_datetime(req.json()['Items'][0]['Items'][-1]['Timestamp']))

TypeError: 'Timestamp' object cannot be interpreted as an integer

In [ ]:
with open('example_json/akers.json', 'w') as outfile:
    json.dump(req.json()['Items'][0]['Items'], outfile)

In [ ]:
timestamps, values, units, goods, questionables, substitutes, annotated = [], [], [], [], [], [], []
for item in req.json()['Items'][0]['Items']:
    timestamps.append(item['Timestamp'])
    values.append(item['Value'])
    units.append()

[{'Timestamp': '2021-11-05T22:40:48.451004Z',
  'Value': 201296.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2021-11-05T22:40:53.498001Z',
  'Value': 201297.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2021-11-05T23:05:28.4260101Z',
  'Value': 201297.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2021-11-05T23:05:33.4110107Z',
  'Value': 201298.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2021-11-05T23:06:37.4730072Z',
  'Value': {'Name': 'Calc Failed', 'Value': 249, 'IsSystem': True},
  'UnitsAbbreviation': '',
  'Good': False,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2021-11

In [ ]:
import pandas as pd

df = pd.read_json('example_json/akers.json')

df.to_csv('example_csv/akers.csv')

In [13]:
req2 = requests.get('https://osi.ipf.msu.edu/piwebapi/assetdatabases/F1RDxYSRoDn9-066ETsCvigyBwjEBIyuXKmUGi-Jb82kTJRwSVBGLVBQSElTVERBVEFcQ0FMQ19URVNU/elements')


In [9]:
import pandas as pd
buildings = pd.read_csv('helper_data/bldg_to_sampleloc.csv')
#buildings.any({'meter_id' : '0331W1'})
'0331W1' in list(buildings['meter_id'])

True

In [33]:
def parse_datetime_with_variable_fraction(datetime_str, format_str):
    fractional_start = datetime_str.index('.') + 1
    fractional_end = datetime_str.index('Z')
    fractional_part = datetime_str[fractional_start:fractional_end]

    # Truncate or pad the fractional part to fit exactly six digits
    adjusted_fractional_part = (fractional_part[:6] + '000000')[:6]

    # Replace the original fractional part with the adjusted one
    adjusted_datetime_str = datetime_str[:fractional_start] + adjusted_fractional_part + datetime_str[fractional_end:]

    dt_object = datetime.strptime(adjusted_datetime_str, format_str)

    return dt_object

    

In [ ]:
def recursive_get(end_date):
    

In [52]:
from datetime import date
from datetime import datetime
start_date = date(2020,1,1)

format_str = "%Y-%m-%dT%H:%M:%S.%fZ"
today = date.today()
interval = (today-start_date).days
meters = list(buildings['meter_id'])
for item in tqdm(req2.json()['Items']):
    req3 = requests.get('https://osi.ipf.msu.edu/piwebapi/elements/{}/elements'.format(item['WebId']))
    for item2 in req3.json()['Items']:
        if item2['Name'] in meters:
            interval_temp = interval
            content = []
            for i in range(0,4):

                recorded = requests.get('https://osi.ipf.msu.edu/piwebapi/streamsets/{}/recorded?StartTime=*-{}d&EndTime=*-{}&MaxCount=50000'.format(item2['WebId'], interval_temp, int(interval / 4 * (3-i))))
                interval_temp = interval / 4 * (4-i)
                cont = recorded.json()['Items'][0]['Items']
                content += cont
            dt_object = parse_datetime_with_variable_fraction(recorded.json()['Items'][0]['Items'][-1]['Timestamp'], format_str)

            # Extract the date from the datetime object
            date_object = dt_object.date()
            while(date.today() - date_object).days > 0:
                
                recorded = requests.get('https://osi.ipf.msu.edu/piwebapi/streamsets/{}/recorded?StartTime=*-{}d&EndTime=*-0d&MaxCount=50000'.format(item2['WebId'], (date.today() - date_object).days))
                content += recorded.json()['Items'][0]['Items']
                dt_object = parse_datetime_with_variable_fraction(recorded.json()['Items'][0]['Items'][-1]['Timestamp'], format_str)
                date_object = dt_object.date()

            with open('example_json/{}_{}.json'.format("_".join(item['Name'].replace('/', '').split(' ')).lower(), item2['Name']), 'w') as outfile:
                try:
                    
                    
                    
                    json.dump(content , outfile)
                    df = pd.read_json('example_json/{}_{}.json'.format("_".join(item['Name'].split(' '), item2['Name']).lower()))
                    df.to_csv('example_csv/{}.{}.csv'.format("_".join(item['Name'].split(' ')).lower(), item2['Name']))
                    
                except:
                    pass
            

100%|██████████| 188/188 [02:24<00:00,  1.30it/s]


In [36]:
from datetime import date
start_date = date(2020,1,1)
today = date.today()
interval = (today-start_date).days
for i in range(0,5):
    #print(interval)
    print(interval / 4 * (3-i))
    #interval = interval / 4 * (4-i)

894.0
596.0
298.0
0.0
-298.0


In [53]:
import pandas as pd
json_dir = os.listdir('example_json')
for json_file in json_dir:
    print(json_file)
    json_data = pd.read_json('example_json/' + json_file)
    json_data.to_csv('example_csv/{}.csv'.format(json_file.split('.')[0]))

holden_hall_0332W1.json
wonders_hall_0323W1.json
wilson_hall_0322W1.json
holmes_hall_0330W1.json
akers_hall_0326W1.json
gilchrist_hall_0307W1.json
mcdonel_hall_0324W1.json
case_hall_0321W1.json
building_1803_1803W1.json
football_building_0077W2.json
building_1802_1802W1.json
building_1808_1808W1.json
fee_hall_0327W1.json
mason_abbot_hall_0302W2.json
butterfield_0310W1.json
emmons_hall_0314W1.json
mason_abbot_hall_0302W1.json
landon_hall_0305W1.json
building_1807_1807W1.json
building_1804_1804W1.json
building_1806_1806W1.json
building_1809_1809W1.json
williams_hall_0308W1.json
armstrong_hall_0316W1.json
hubbard_hall_0331W1.json
yakeley_hall_0306W1.json
bryan_hall_0312W1.json
psychology_0027W1.json
building_1805_1805W1.json
football_building_0077W1.json
building_1810_1810W1.json
snyder-phillips_hall_0300W1.json


In [4]:
import os
import pandas as pd
csv_dir = os.listdir('meters_csv')
for csv_file in csv_dir:
    #print(csv_file)
    print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
    print(csv_file)
    

2023-04-07 07:49:20.220001200+00:00
emmons_hall_0314W1.csv
2023-04-07 07:48:25.074005100+00:00
fee_hall_0327W1.csv
2023-04-07 07:46:45.088012600+00:00
armstrong_hall_0316W1.csv
2023-04-07 07:43:30.033004700+00:00
building_1810_1810W1.csv
2023-04-07 07:46:20.371002100+00:00
wilson_hall_0322W1.csv


/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])


2023-04-07 07:43:10.426010100+00:00
mason_abbot_hall_0302W1.csv
2023-04-07 07:49:20.028015100+00:00
building_1809_1809W1.csv
2023-04-07 07:49:25.732009800+00:00
mason_abbot_hall_0302W2.csv


/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])


2023-04-07 07:46:40.471008300+00:00
case_hall_0321W1.csv
2023-04-07 07:49:00.515014600+00:00
building_1804_1804W1.csv
2023-04-07 07:49:05.598007200+00:00
building_1803_1803W1.csv
2023-04-07 07:46:20.311004600+00:00
landon_hall_0305W1.csv


...

/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])


2023-04-07 07:49:05.278015100+00:00
building_1805_1805W1.csv
2023-04-07 07:43:05.743011400+00:00
building_1802_1802W1.csv
2023-04-07 07:45:30.497009200+00:00
football_building_0077W1.csv


/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])
/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])


2023-04-07 07:48:05.387008600+00:00
wonders_hall_0323W1.csv
2023-04-07 07:48:45.718002300+00:00
building_1806_1806W1.csv
2023-04-07 07:46:55.043014500+00:00
football_building_0077W2.csv
2023-04-07 07:49:10.448013300+00:00
holmes_hall_0330W1.csv
2023-04-07 07:49:00.429000800+00:00
hubbard_hall_0331W1.csv
2023-04-07 07:50:00.460006700+00:00
yakeley_hall_0306W1.csv
2023-04-07 07:48:30.577011100+00:00
building_1807_1807W1.csv


/tmp/ipykernel_366409/3372402382.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv('meters_csv/' + csv_file).iloc[-1]['Timestamp'])


array(['1801', '1802', '1803', '1804', '1805', '1806', '1807', '1808',
       '1809', '1810', 'abbot', 'akers', 'armstrong', 'bryan',
       'butterfield', 'case', 'duffy', 'emmons', 'fee', 'gilchrist',
       'holden', 'holmes', 'hubbard', 'landon', 'mcdonel', 'psychology',
       'snyder phillips', 'williams', 'wilson', 'wonders', 'yakeley'],
      dtype=object)

In [89]:
csv_dir = os.listdir('example_csv')
for csv_file in csv_dir:
    print(csv_file.split('_')[0].lower())
    if csv_file.strip('_')[0] not in np.array(bldg_sample_loc.building.str.lower()):
        #os.remove('example_csv/{}'.format(csv_file))
        #print('Removed: {}'.format(csv_file))
        pass


1
a
a
a
a
a
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
c
c
c
c
c
c
c
c
c
c
c
d
d
e
e
e
e
f
f
f
f
f
f
f
f
f
f
f
f
f
f
f
g
h
h
h
h
i
i
i
i
i
i
i
j
k
k
k
k
l
l
l
l
l
l
l
l
m
m
m
m
m
m
m
m
m
m
m
m
m
m
n
n
n
o
o
o
o
o
o
p
p
p
p
p
p
p
p
p
p
r
r
r
r
s
s
s
s
s
s
s
s
s
s
s
t
t
t
u
u
u
v
v
v
v
v
v
w
w
w
w
w
w
w
y


In [47]:
id_ = 'F1EmxYSRoDn9-066ETsCvigyBwUY7hpJ0n7BGtB-RU6E1VeASVBGLVBQSElTVERBVEFcQ0FMQ19URVNUXEZFRSBIQUxMXDAzMjdXMQ'
recorded = requests.get('https://osi.ipf.msu.edu/piwebapi/streamsets/{}/recorded?StartTime=*-{}d&EndTime=*-1d&MaxCount=50000'.format(id_, interval))


[{'Timestamp': '2020-01-02T13:31:44Z',
  'Value': 5583.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2020-01-02T13:32:04Z',
  'Value': 5584.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2020-01-02T14:44:24Z',
  'Value': 5584.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2020-01-02T14:44:44Z',
  'Value': 5585.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2020-01-02T16:27:04Z',
  'Value': 5585.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questionable': False,
  'Substituted': False,
  'Annotated': False},
 {'Timestamp': '2020-01-02T16:27:24Z',
  'Value': 5586.0,
  'UnitsAbbreviation': 'Kgal',
  'Good': True,
  'Questio

In [49]:
type(recorded.json()['Items'][0]['Items'][-1]['Timestamp'])

str

In [29]:
from datetime import datetime
datetime_str = "2021-11-05T22:40:48.451004Z"

# Parse the datetime string into a datetime object
dt_object = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

# Extract the date from the datetime object
date_object = dt_object.date()

print((date_object - date.today()).days)

-518


In [1]:
import json
existing_data = json.load('meters_json/akers_hall_0326W1.json')

NameError: name 'json' is not defined